In [8]:
import pandas as pd

gs_res = pd.read_csv('project1_grid_search.csv', index_col='Unnamed: 0').sort_values(by=['val_auc'], ascending=False)
#0.829998
gs_res.head()

,bagging_fraction,bagging_freq,bagging_seed,best_round,boosting,categorical_column,cv,feature_fraction,feature_fraction_seed,gs_timespent,...,num_threads,objective,pred_timespent,preds,scale_pos_weight,timespent,timestamp,train_auc,val_auc,verbose
1529021613,1.0,1,16455427,2448,gbdt,"""[528, 536, 726, 728, 867, 1158, 1169, 1474]""",True,0.7,16455427,NaN,...,4,binary,NaN,NaN,2,NaN,2018-06-14 20:13:33,NaN,0.829998,1
1529016426,0.6,1,31060497,167,gbdt,"""[528, 536, 726, 728, 867, 1158, 1169, 1474]""",True,1.0,31060497,NaN,...,4,binary,NaN,NaN,2,NaN,2018-06-14 18:47:06,NaN,0.819533,1
1529018698,0.4,1,41499713,1720,gbdt,"""[528, 536, 726, 728, 867, 1158, 1169, 1474]""",True,0.3,41499713,NaN,...,4,binary,NaN,NaN,2,NaN,2018-06-14 19:24:58,NaN,0.817727,1
1529038836,0.9,1,4863757,4000,gbdt,"""[528, 536, 726, 728, 867, 1158, 1169, 1474]""",True,1.0,4863757,NaN,...,4,binary,NaN,NaN,2,NaN,2018-06-15 01:00:36,NaN,0.809534,1
1529079354,0.9,1,38016846,40,gbdt,"""[137, 375, 483, 491, 560, 718, 739, 811]""",True,0.6,38016846,0:09:08,...,4,binary,NaN,NaN,2,NaN,2018-06-15 12:15:54,NaN,0.798698,1


In [9]:
#gs_res.gs_timespent
gs_res.pred_timespent

1529021613        NaN
1529016426        NaN
1529018698        NaN
1529038836        NaN
1529079354        NaN
1529078964        NaN
1529080397    0:01:21
1529080656    0:01:10
1529005138        NaN
1529078391        NaN
1529014439        NaN
1529014419        NaN
1529014400        NaN
1529005975        NaN
1529014454        NaN
1529007956        NaN
1529006206        NaN
1529014393        NaN
1529014412        NaN
1529014409        NaN
1529007953        NaN
1529005557        NaN
1529014159        NaN
1529006204        NaN
1529014405        NaN
1529005829        NaN
1529014458        NaN
1529005980        NaN
1529010730        NaN
1529014416        NaN
1529014398        NaN
1529010729        NaN
1529014162        NaN
1529005826        NaN
1529014434        NaN
Name: pred_timespent, dtype: object